In [ ]:
# importing attention layer from external package
import attention
from attention import AttentionLayer


import pandas as pd
import numpy as np
import re
import nltk
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences 
from nltk.corpus import stopwords



from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Attention


In [ ]:
# COnfiguration For Training the Model on GPU with CUDA

from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

devices = get_available_devices()
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

['/device:CPU:0', '/device:XLA_CPU:0', '/device:GPU:0', '/device:XLA_GPU:0']
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


#  Import the data from file

In [ ]:
# We are taking only 100000 rows of the data set for training
with tf.device(devices[2]):
    df = pd.read_csv('../archive/Reviews.csv',nrows = 200000)

In [ ]:
# removing Null, Duplicates rows from data frame
with tf.device(devices[2]):
    df.isnull().sum()
    df.drop_duplicates(subset=['Text'],inplace=True)
    df = df[df['Summary'].notna()]
    df.head()

In [ ]:
# Keep Only two rows from the data frame and dropping all others
with tf.device(devices[2]):
    df=df[['Summary','Text']]

In [ ]:
# Make dictionary of contractions words of the english 
with tf.device(devices[2]):
    contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because",
                    "could've": "could have", "couldn't": "could not","didn't": "did not",  "doesn't": "does not",
                    "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                    "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                    "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have",
                    "I'm": "I am", "I've": "I have", "i'd": "i would",
                    "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am",
                    "i've": "i have", "isn't": "is not", "it'd": "it would",
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's":
                    "it is", "let's": "let us", "ma'am": "madam",
                    "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've":
                    "might not have", "must've": "must have","mustn't": "must not", "mustn't've":
                    "must not have", "needn't": "need not","needn't've":
                    "need not have","o'clock": "of the clock",
                    "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                    "sha'n't": "shall not", "shan't've": "shall not have",
                    "she'd": "she would", "she'd've": "she would have", "she'll": "she will",
                    "she'll've": "she will have", "she's": "she is",
                    "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                    "so've": "so have","so's": "so as",
                    "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                    "there'd've": "there would have", "there's": "there is", "here's": "here is",
                    "they'd": "they would", "they'd've": "they would have",
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are",
                    "they've": "they have", "to've": "to have",
                    "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                    "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
                    "what're": "what are",
                    "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                    "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                    "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                    "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                    "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                    "you're": "you are", "you've": "you have"}

# Pre-processing 

In [ ]:
with tf.device(devices[2]):
    
    summary = df['Summary']
    text = df['Text']

    def func(text, flag):
        # text is the original summary and original text data which will be input.
        # If Flag is 'S' then it's summary and we won't remove stop words from that otherwise we'll remove Stop words from the text
        
        # convert to lower case 
        text = text.apply(lambda x: x.lower())
        
        # remove contractions
        for i in range(len(text)):
            try:
                sent = text[i]
                words = []
                for j in sent.split(" "):
                    if j in contractions:
                        words.append(contractions[j])
                    else:
                        words.append(j)
                text[i] = ' '.join(words)
            except Exception:
                pass

        # remove numbers, unwanted space, punctuations etc. 
        text = text.apply(lambda x: re.sub("[^a-zA-Z]", " ", x))
        text = text.apply(lambda x: re.sub('[m]{2,}', 'mm', x))
        text = text.apply(lambda x: x.replace('[^\w\s]',''))
        text = text.apply(lambda x: re.sub(r'https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE))
        text = text.apply(lambda x: re.sub(r'\<a href', ' ', x))
        text = text.apply(lambda x: re.sub(r'&amp;', '', x))
        text = text.apply(lambda x: re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', x))
        text = text.apply(lambda x: re.sub(r'<br />', ' ', x))
        text = text.apply(lambda x: re.sub(r'\'', ' ', x))


        # making set of all stop words
        stop = set(stopwords.words('english'))
        
        # remove stop words from text
        if flag != 'S':
            text= text.apply(lambda x:' '.join([word for word in x.split(' ') if word not in (stop)]))


        text = text.apply(lambda x:re.sub("[\(\[].*?[\)\]]", "", x))
        text = text.apply(lambda x:re.sub(' +', ' ',x))

        return text




    # calling func fro text 
    text = func(text, 'T')


    # calling func fro Summary
    summary = func(summary, 'S')

    
    # making data frame again
    df['Summary'] = summary
    df['Text'] = text


In [ ]:
# Printing Cleaned Summary
with tf.device(devices[2]):
    print("Original Reviews")
    for i in range(10):
        print(str(i) + " " + df['Text'][i])
        
    print()
    print("Original Summaries")
    for i in range(10):
        print(str(i) + " " + df['Summary'][i])



Original Reviews
0 bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better 
1 product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo 
2 confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story c lewis lion witch wardrobe treat seduces edmund selling brother sisters witch 
3 looking secret ingredient robitussin believe found got addition root beer extract ordered good made cherry soda flavor medicinal 
4 great taffy great price wide assortment yummy taffy delivery quick taffy lover deal 
5 got wild hair taffy ordered five pound bag taffy enjoyable many flavors watermelon root beer melon peppermint grape etc complaint bit much red black licorice flavored pi

In [ ]:
with tf.device(devices[2]):
    df.replace('', np.nan, inplace=True)
    df.dropna(axis=0,inplace=True)

In [ ]:
# Counting Summaries with length less than 8 and Text with length less than 50
with tf.device(devices[2]):
    print(len(df))

    count = 0
    for i in df['Summary']:
        if(len(i.split())<=8):
            count += 1
    print("Total rows with summary length <=8 are = ", count)
    print(count/len(df['Summary']))

    
###### idhr change kra h 80 50 ki jagah (Kr skte h ek bar train 80 pr bhi )

    count = 0
    for i in df['Text']:
        if(len(i.split())<=50):
            count += 1
    print("Total rows with Text length <=50 are = ", count)
    print(count)
    print(count/len(df['Text']))

162841
Total rows with summary length <=8 are =  151690
0.9315221596526674
Total rows with Text length <=50 are =  122859
122859
0.7544721538187558


In [ ]:
# Decarding rows which has summary or Text length below or above the threesold

with tf.device(devices[2]):

    sorted_summary = []
    sorted_text = []
    min_summary_length = 1
    max_summary_length = 8
    min_text_length = 5
    max_text_length = 50
    

    for i in range(len(text)):
        try:

            text_length = len(text[i].split(' '))
            summary_length = len(summary[i].split(' '))

            if((text_length >= min_text_length) and (text_length<=max_text_length)
              and summary_length >= min_summary_length and summary_length<=max_summary_length):
                sorted_summary.append(summary[i])
                sorted_text.append(text[i])
        except Exception:
            pass


    summary = sorted_summary
    text = sorted_text

    print(len(summary))

96706


In [ ]:
# Adding start and end tokens in the summary (It will be used in Decoding the output of the sequence in last step)

with tf.device(devices[2]):
    df = pd.DataFrame({'Text' : text, 'Summary' : summary})


    df['Summary'] = df['Summary'].apply(lambda x : 'strtok ' + x + ' endtok')
    summary = df['Summary']

In [ ]:
# counting the number of Unique words in both text and summaries and also counting rare words with freequency less tham threshold

def count_words(text):
    words = dict()
    for sentence in text:
        for curr in sentence.split(' '):
            if curr in words:
                words[curr]+=1
            else:
                words[curr] = 1
    return words

def rare_count_words( words, threshold ):
    rare_words = 0
    for word in words:
        if words[word]<threshold :
            rare_words = rare_words+1
    return rare_words  


with tf.device(devices[2]):

    unique_text_words = count_words(text)
    unique_summary_words = count_words(summary)

    unique_text_words_count = len(unique_text_words)
    unique_summary_words_count = len(unique_summary_words)


    # count_words is total number of unique words 

    # rare words are total words which are below threshold

    text_threshold = 4
    summary_threshold = 6

    rare_text_words_count = rare_count_words(unique_text_words,text_threshold)
    rare_summary_words_count = rare_count_words(unique_summary_words, summary_threshold)



    print(" Total Unique words in Text : ", unique_text_words_count)
    print(" Total Unique words in Summary : ", unique_summary_words_count)
    print(" Total rare words in Text : ", rare_text_words_count)
    print(" Total rare words in Summary : ", rare_summary_words_count)


 Total Unique words in Text :  40714
 Total Unique words in Summary :  14362
 Total rare words in Text :  26079
 Total rare words in Summary :  10982


# Tokenizing the text to convert words into integers

In [ ]:
with tf.device(devices[2]):
    
    # create tokenizer for Text and we will be assigning integers only to important word(i.e descard rare words)
    # num_words argument in Toeknizer function will discard rare words 
    
    text_tokenizer = Tokenizer(num_words  = (unique_text_words_count - rare_text_words_count))
    text_tokenizer.fit_on_texts(text)

    # this will be Interger sequences of all the text data 
    text_sequences = text_tokenizer.texts_to_sequences(text)
    text_data = pad_sequences(text_sequences, maxlen= max_text_length , padding= 'post')

    text = text_data



    # create tokenizer for Text and we will be assigning integers only to important word(i.e descard rare words)
    # num_words argument in Toeknizer function will discard rare words 

    
################## + 2 mi ayega idhr 

    summary_tokenizer = Tokenizer(num_words  = unique_summary_words_count - rare_summary_words_count + 2)
    summary_tokenizer.fit_on_texts(summary)

    # this will be Interger sequences of all the summaries
    summary_sequences = summary_tokenizer.texts_to_sequences(summary)
    summary_data = pad_sequences(summary_sequences, maxlen= max_summary_length , padding= 'post')

    summary = summary_data


    # text_vocab_length will be total words present into text after discarding rare words and same with summary_vocab_length
    
    text_vocab_length = text_tokenizer.num_words + 1
    summary_vocab_length = summary_tokenizer.num_words + 1


    print(text_vocab_length)
    print(summary_vocab_length)





14636
3383


In [ ]:
# Deleting the rows which has only 2 words (i.e. strtok and endtok) beacuase there may be some summaries 
# with only rare words and that may become empty after tokenize

with tf.device(devices[2]):
    indexes=[]
    for i in range(len(summary)):
        count=0
        for curr_value in summary[i]:
            if curr_value != 0:
                count=count+1
        if(count==2):
            indexes.append(i)

    # delete the rows which are there in indexes list 
    summary =np.delete(summary,indexes, axis=0)
    text=np.delete(text,indexes, axis=0)

In [ ]:
from keras import backend as K 
K.clear_session()

# total number of LSTM cells in one layer of LSTM
LSTM_CELLS_COUNT = 500

# Length of feature vector/ total features considered for each word in the embedding matrix
TEXT_EMBEDDING_DIM = 300


with tf.device(devices[2]):
    
    # Shape of encoder Inputs 
    encoder_inputs = Input(shape=(max_text_length,))
    
    # Embedding matrix for encoder which gives input to the encoder according to input vector
    encoder_embedding_layer = Embedding(text_vocab_length, TEXT_EMBEDDING_DIM,trainable=True)(encoder_inputs)

    
    # Stacked LSTM layers 

    # encoder layer 1 which will corresponding embedding matrix as a input 
    encoder_lstm_layer1 = LSTM(LSTM_CELLS_COUNT, return_sequences=True, return_state=True,  dropout = 0.5,activation = 'tanh',recurrent_activation = 'sigmoid',recurrent_dropout = 0,unroll = False,use_bias = True)
    # Outputs of LSTM layer1
    encoder_output1, state_h_layer1, state_c_layer1 = encoder_lstm_layer1(encoder_embedding_layer)

    # encoder layer 2
    encoder_lstm_layer2 = LSTM(LSTM_CELLS_COUNT, return_sequences=True, return_state=True, dropout = 0.5,activation = 'tanh',recurrent_activation = 'sigmoid',recurrent_dropout = 0,unroll = False,use_bias = True)
    # Outputs of LSTM layer2
    encoder_output2, state_h_layer2, state_c_layer2 = encoder_lstm_layer2(encoder_output1)

    # encoder layer 3
    encoder_lstm_layer3 = LSTM(LSTM_CELLS_COUNT, return_sequences=True, return_state=True, dropout = 0.5,activation = 'tanh',recurrent_activation = 'sigmoid',recurrent_dropout = 0,unroll = False,use_bias = True )
    # Outputs of LSTM layer1
    encoder_output3, state_h_layer3, state_c_layer3 = encoder_lstm_layer3(encoder_output2)


    #decoder 
    decoder_inputs = Input(shape=(None,))
    
    # Embedding Matrix of decoder 
    embedding_layer_decoder = Embedding(summary_vocab_length, TEXT_EMBEDDING_DIM,trainable=True)
    decoder_embedding = embedding_layer_decoder(decoder_inputs)

    
    # LSTM layer of decoder layer 
    decoder_lstm = LSTM(LSTM_CELLS_COUNT, return_sequences=True, return_state=True, dropout = 0.5,activation = 'tanh',recurrent_activation = 'sigmoid',recurrent_dropout = 0,unroll = False,use_bias = True)
    decoder_output, fw_state, bw_state = decoder_lstm(decoder_embedding, initial_state = [state_h_layer3, state_c_layer3] )

    #attention layer

    attn_layer = AttentionLayer(name='attention_layer')
    attention_out, x = attn_layer([encoder_output3, decoder_output])

    
    decoder_attention_concat = Concatenate(axis = -1, name = "concat_layer")([decoder_output, attention_out])


    # dense layer to apply softmaxx activation function and its outputs will be final outputs 

    decoder_dense = TimeDistributed(Dense(summary_vocab_length, activation = 'softmax'))
    decoder_output = decoder_dense(decoder_attention_concat)


    model = Model( [encoder_inputs, decoder_inputs],  decoder_output)

    model.summary()









Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 300)      4390800     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 50, 500), (N 1602000     embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
_______________________________________________________________________________________

In [ ]:
# compile the model with correct optimizer and loss function

with tf.device(devices[2]):
    model.compile(optimizer = 'rmsprop', loss = 'sparse_categorical_crossentropy')
    early_stopping = EarlyStopping(monitor='val_loss', mode = 'min',  patience=2, verbose=1)

# Split the data into training and testing parts

In [ ]:
# Spilt tha data into test and train sets
with tf.device(devices[2]):
    text_train, text_test, summary_train, summary_test = train_test_split(np.array(text), np.array(summary), test_size = 0.1, random_state = 0, shuffle = True)

In [ ]:
with tf.device(devices[2]):
    print(text_train[:10])
    print(summary_train[:10])

[[ 190  811   30   31  197  544 1128  176  235    3  622  104 1303  209
    11    6    4   40  552  149  688  633 1803  211  817  441  132  176
    46    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [   8 1723   46   49 1502   25   20    7   68  491  857   78  184 1677
    62   33   46   49  297  809   46   49  297    7 6153   10   17 1395
    16   13    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [   4    5   36  656   16  191  160   70   40    2  340 2542  126    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]
 [  25  210 1186 3624 1022  869  177    2    9 1468  639 3703  284  634
    71  869 2260 3693 2962    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0 

In [ ]:
# # Load the assets and values saved in the saved model
# Un comment this when you have trained your model and want to use layers of that and specify the file name in last line of this cell

# from tensorflow.keras.models import load_model

# model = tf.keras.Sequential()
# with tf.device(devices[2]):
#     model = load_model("new_model_7.h5",custom_objects={'AttentionLayer': AttentionLayer})

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 300)      4390800     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 50, 500), (N 1602000     embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
_______________________________________________________________________________________

In [ ]:
# Fitting the model on the training set
with tf.device(devices[2]):
    # trained_model = model.fit([text_train, summary_train, summary_train, epochs=15,callbacks=[early_stopping],batch_size=128, validation_data=([text_test,summary_test], summary_test)    
    trained_model = model.fit([text_train, summary_train[:,:-1]], summary_train.reshape(summary_train.shape[0],summary_train.shape[1], 1)[:,1:], epochs=15,callbacks=[early_stopping],batch_size=128, validation_data=([text_test,summary_test[:,:-1]], summary_test.reshape(summary_test.shape[0],summary_test.shape[1], 1)[:,1:]))

Epoch 1/15
669/669 [==============================] - 455s 681ms/step - loss: 2.8459 - val_loss: 2.5538
Epoch 2/15
669/669 [==============================] - 454s 679ms/step - loss: 2.4481 - val_loss: 2.3722
Epoch 3/15
669/669 [==============================] - 454s 679ms/step - loss: 2.2997 - val_loss: 2.2920
Epoch 4/15
669/669 [==============================] - 455s 680ms/step - loss: 2.2022 - val_loss: 2.2324
Epoch 5/15
669/669 [==============================] - 456s 682ms/step - loss: 2.1272 - val_loss: 2.2026
Epoch 6/15
669/669 [==============================] - 455s 680ms/step - loss: 2.0643 - val_loss: 2.1765
Epoch 7/15
669/669 [==============================] - 457s 683ms/step - loss: 2.0113 - val_loss: 2.1629
Epoch 8/15
669/669 [==============================] - 456s 682ms/step - loss: 1.9642 - val_loss: 2.1569
Epoch 9/15
669/669 [==============================] - 463s 692ms/step - loss: 1.9190 - val_loss: 2.1578
Epoch 10/15
669/669 [==============================] - 455s 680m

In [ ]:
with tf.device(devices[2]):
    model.save("new_model_11.h5")   # enter the file name to save your model

# Inference

In [ ]:
# Getting values back from saved models (Layers those were saved after training of the model)

from keras import backend as K 
K.clear_session()

with tf.device(devices[2]):
    
    # Unpacking the loaded model 
    
    # Getting the LSTM layers from the models 
    
    encoder_inputs = Input(shape=(max_text_length,))
    encoder_embedding_layer = model.get_layer('embedding')(encoder_inputs)
    
    # encoder layer 1
    encoder_lstm_layer1 = model.get_layer('lstm')
    encoder_output1, state_h_layer1, state_c_layer1 = encoder_lstm_layer1(encoder_embedding_layer)

    # encoder layer 2
    encoder_lstm_layer2 = model.get_layer('lstm_1')
    encoder_output2, state_h_layer2, state_c_layer2 = encoder_lstm_layer2(encoder_output1)

    # encoder layer 3
    encoder_lstm_layer3 = model.get_layer('lstm_2')
    encoder_output3, state_h_layer3, state_c_layer3 = encoder_lstm_layer3(encoder_output2)




In [ ]:

with tf.device(devices[2]):
    
    # encoder model
    encoder_model = Model(inputs = encoder_inputs, outputs = [encoder_output3, state_h_layer3, state_c_layer3])

    decoder_state_input_h = Input(shape=(LSTM_CELLS_COUNT,))
    decoder_state_input_c = Input(shape=(LSTM_CELLS_COUNT,))
    decoder_hidden_state_input = Input(shape=(max_text_length,LSTM_CELLS_COUNT))

    decoder_embedding_2 = model.get_layer('embedding_1')(decoder_inputs)
    decoder_output_2 , decoder_state_h_2, decoder_state_c_2 = model.get_layer('lstm_3')(decoder_embedding_2, initial_state = [decoder_state_input_h, decoder_state_input_c])

    # attention inference

    attention_output_inference, attention_states_infernce = model.get_layer('attention_layer')([decoder_hidden_state_input, decoder_output_2])
    decoder_inference_concat = Concatenate(axis=-1, name='concat')([decoder_output_2, attention_output_inference])

    # dense layer  

    decoder_output_2 = model.get_layer('time_distributed')(decoder_inference_concat)

    # decoder model

    decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c], 
                         [decoder_output_2 ] + [decoder_state_h_2, decoder_state_c_2],name='decoder_output_p1')



# Decode the sequence 

In [ ]:
# This reverse Dictionaries will have word as a value for given number 
text_int_to_word = text_tokenizer.index_word
summary_int_to_word = summary_tokenizer.index_word
word_index_summary= summary_tokenizer.word_index

In [ ]:
# convert sequences of integers of summaries to original summaries

def sequence_to_summary(input_sequence):
    answer = ''

    start  = word_index_summary['strtok']
    end  = word_index_summary['endtok']

    for number in input_sequence:
        if(number!=0 and number != start and number != end):
            answer += summary_int_to_word[number] + ' '
    return answer

In [ ]:
# convert sequences of integers of summaries to original summaries

def sequence_to_text(input_sequence):
    answer = ''
    for number in input_sequence:
        if(number!=0):
            answer += text_int_to_word[number] + ' '
    return answer

In [ ]:
# decode the sequence for answer
# target[0,0] == word_index_summary['endtok']
def decode_the_sequence(vec):

    encoder_output, state_h , state_c = encoder_model.predict(vec)

#     print("here =",encoder_output)
    target = np.zeros(shape = (1,1))

    decode_sum = ''

    count = 0
    target[0,0] =  word_index_summary['strtok']

    while not ( target[0,0] == word_index_summary['endtok'] or count > max_summary_length):

        decoder_output, decoder_state_h, decoder_state_c = decoder_model.predict([target] + [encoder_output, state_h , state_c]) 
        
#         print(decoder_output)
        
        next_index = np.argmax(decoder_output[0, -1, :])
#         if next_index ==0:
#             next_index=5
        if next_index == word_index_summary['endtok']:
            break
#         print(next_index)
        decode_sum+=summary_int_to_word[next_index] + ' '

#         print(decode_sum)

        target[0,0] = next_index

        state_h , state_c = decoder_state_h, decoder_state_c
        count+=1




    return decode_sum



In [ ]:
for i in range(1950,2050):
#     print(text_train[i])
    print("Review:",sequence_to_text(text_test[i]))
    print("Original summary:",sequence_to_summary(summary_test[i]))
    print("Predicted summary:",decode_the_sequence(text_test[i].reshape(1,max_text_length)))
#      print("Predicted new model:",decode_the_sequence(text_train[i].reshape(1,max_text_length)))
#     print("\n")

Review: taste good br price good br br upset stomach one br healthy comparison products really healthy br br bars good quick snack fresh food available let us face even kashi adds lot artificial ingredients keep flavor preservation 
Original summary: decent bars 
Predicted summary: great product 
Review: throw piece junk year service warranty nobody area second time breaks first time fixed made pay shipping charges ended expensive buy box packaging materials broke nothing terrible product even worse customer service 
Original summary: a lemon do not waste your money 
Predicted summary: do not buy 
Review: hell red best commercially available salsa market tastes fresh packs ton flavor prefer mild husband likes hot perfect adding special dishes e g guacamole dip king ranch chicken casserole etc 
Original summary: best salsa 
Predicted summary: the best 
Review: usually big fan herbal teas beverages seriously disgusting drink could would brewed actually take sip smelling odor even five ye

Predicted summary: great product 
Review: favorite granola hands weird processed taste like many foods days prefect combination chewy crunchy sweet plus healthy snack vanilla yogurt meal 
Original summary: best granola ever 
Predicted summary: delicious 
Review: toy great able car seat stroller son loves play things grab hold different colors textures make great time 
Original summary: great toy 
Predicted summary: my dog loves these 
Review: way better hot sauce ordered past thank carrying product 
Original summary: texas hot sauce 
Predicted summary: hot hot hot 
Review: bought product reading reviews elsewhere see improvement using cream use morning anything eyes seem swollen sorry 
Original summary: no results 
Predicted summary: not as good as i wanted 
Review: pearls boil perfect every time even better boba drinks get boba tea shops highly recommended 
Original summary: perfect every time 
Predicted summary: perfect for a 
Review: greyhounds go absolutely bananas chews give rawhi

Predicted summary: great product 
Review: completely agree previous review outstanding coffee tried lot k cup varieties found one best wonderfully smooth flavor really superior cup coffee 
Original summary: great coffee 
Predicted summary: best coffee ever 
Review: love taste dont know drink bad reviews personally like much n e coconut water 
Original summary: best coconut water i have ever had 
Predicted summary: tastes like plastic 
Review: originally unhappy order appeared dead complain got back sent replacement gave trouble culture grew well since several months well pleased defense pennsylvania earthquake strange weather first go around 
Original summary: easy to use 
Predicted summary: great for cooking 
Review: purchased item anxious see results eyes dark puffy received product excited realized pump work squeezed cream seems work fine however seen results yet going previous reviews reason purchased product 
Original summary: pump does not work 
Predicted summary: not as advertis

In [ ]:
# from rouge_score import rouge_scorer
# # # scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
# # # scores = scorer.score('The quick brown fox jumps over the lazy dog',
# # #                       'The quick brown dog jumps on the log.')
# for i in range(100):
#     scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

#     scores = scorer.score(o[i],p[i])
#     print("\n",scores)
